In [1]:
# import, authenticate and initialize ee
import ee
ee.Initialize()

In [2]:
# import necessary libraries
import geemap

# Polygon Geometry

In [3]:
# Irregular polygon over India to filter Images
geometry = ee.Geometry.Polygon(
                  [[[72.45603597810654, 17.04455115190426],
                    [77.02634847810654, 7.524078790515349],
                    [78.52048910310654, 8.394509570247047],
                    [80.22505844476446, 10.615496976585325],
                    [80.75240219476446, 15.1550871492137],
                    [88.92623031976446, 21.57390132315418],
                    [87.21179337029089, 25.773268726099232],
                    [80.00476212029089, 28.35571907813202],
                    [77.98327774529089, 31.852446824865947],
                    [75.61023087029089, 31.852446824865947],
                    [72.76096497130419, 28.62473550203735],
                    [71.88205872130419, 24.298734950710625],
                    [68.19065247130419, 22.767765784818916]]])

In [4]:
Map = geemap.Map()

In [5]:
Map.addLayer(geometry, {}, 'Filter Geometry')
Map.centerObject(geometry)

# Filter Images

In [6]:
# import Google Dynamic World Test Train Dataset and Filter to Geometry
ic = ee.ImageCollection(
    'projects/wri-datalab/dynamic_world/v1/DW_LABELS'
    ).filterBounds(geometry)

In [7]:
# Start and End date of the collection
print(ic.sort(
    'system:time_start',True
    ).first().date().format('Y-M-d').getInfo())
print(ic.sort(
    'system:time_start',False
    ).first().date().format('Y-M-d').getInfo())

2018-12-16
2019-10-22


In [8]:
# filter season wise images and print their respective number of images in the season
kharif = ic.filterDate('2019-06-01', '2019-09-30');
rabi = ic.filter(ee.Filter.Or(
              ee.Filter.date('2018-12-16', '2019-01-31'),
              ee.Filter.date('2019-10-01', '2019-10-31')));
zaid = ic.filterDate('2019-02-01', '2019-05-31')
print(f'{kharif.size().getInfo()} images in Kharif')
print(f'{rabi.size().getInfo()} images in Rabi')
print(f'{zaid.size().getInfo()} images in Zaid')

14 images in Kharif
154 images in Rabi
423 images in Zaid


# Water Class

## Export Water Class Image

In [9]:
# extract the water class - lulc label = 1
water_class = ic.mosaic().select(['lulc']).eq(1)
water_class = water_class.updateMask(water_class)

In [10]:
# export water pixels as Image to EE Asset
task = ee.batch.Export.image.toAsset(
  image = water_class,
  description = 'Water Class',
  assetId = 'users/balakumaran247/swSentinel/water_class_image',
  crs = 'EPSG:4326',
  scale = 10,
  region = geometry,
  maxPixels = 1e13
)

In [11]:
# task.start()

In [12]:
water_class_image = ee.Image("users/balakumaran247/swSentinel/water_class_image")

In [13]:
# Set visualization parameters.
vis_params = {
    "min": 0,
    "max": 1,
    "palette": ["white", "red"],
}

# add the layer to Map
Map.addLayer(water_class_image, vis_params, "Water Class Image", False)

## Sampling Water

In [14]:
# conver to vector
water_vector = water_class_image.reduceToVectors(
  reducer = ee.Reducer.countEvery(),
  geometry = water_class_image.geometry(),
  scale = 10,
  maxPixels = 1e14,
)

In [15]:
# sample water class
water_samples = ee.FeatureCollection.randomPoints(
    region = water_vector,
    points = 800,
    seed = 1234,
    maxError = 1)

In [16]:
# export the Water class Sample points to EE Asset
task = ee.batch.Export.table.toAsset(
  collection = water_samples,
  description = 'Water Samples',
  assetId = 'users/balakumaran247/swSentinel/water_samples',
)

In [17]:
# task.start()

In [18]:
water = ee.FeatureCollection('users/balakumaran247/swSentinel/water_samples')

# add the layer to Map
Map.addLayer(water, {"color": "blue"}, "Water Samples")

# Non Water Class

## Export Non Water Class Image

In [19]:
# extract the non water class - lulc label > 1
non_water_class = ic.mosaic().select(['lulc']).gt(1)
non_water_class = ic.mosaic().updateMask(non_water_class)

In [20]:
# export water pixels as Image to EE Asset
task = ee.batch.Export.image.toAsset(
  image = non_water_class,
  description = 'Non Water Class',
  assetId = 'users/balakumaran247/swSentinel/non_water_class_image',
  crs = 'EPSG:4326',
  scale = 10,
  region = geometry,
  maxPixels = 1e13
)

In [21]:
# task.start()

In [22]:
non_water_class_image = ee.Image("users/balakumaran247/swSentinel/non_water_class_image")

In [23]:
# Set visualization parameters.
vis_params = {
    "bands": ["lulc"],
    "min": 0,
    "max": 1,
    "opacity": 1,
    "palette": ["white", "red"],
}

# add the layer to Map
Map.addLayer(non_water_class_image, vis_params, "Non Water Class Image", False)

## Sampling Non Water

In [24]:
# conver to vector
def class_vector(classno):
  class_vector = non_water_class_image.reduceToVectors(
    reducer = ee.Reducer.countEvery(),
    geometry = non_water_class_image.geometry(),
    scale = 10,
    maxPixels = 1e14,
  )
  # sample water class
  return ee.FeatureCollection.randomPoints(
    region = class_vector,
    points = 100,
    seed = 1234,
    maxError = 1)

In [25]:
non_water_samples = ee.FeatureCollection(ee.List.sequence(2, 9).map(class_vector))

In [26]:
# export the Non Water class Sample points to EE Asset
task = ee.batch.Export.table.toAsset(
  collection = non_water_samples.flatten(),
  description = 'Non Water Samples',
  assetId = 'users/balakumaran247/swSentinel/non_water_samples',
)

In [27]:
# task.start()

In [28]:
non_water = ee.FeatureCollection('users/balakumaran247/swSentinel/non_water_samples')

# add the layer to Map
Map.addLayer(non_water, {"color": "red"}, "Non Water Samples")

In [29]:
# display the map
Map

Map(center=[19.69101674305843, 78.55844139553432], controls=(WidgetControl(options=['position', 'transparent_b…